In [45]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/urdu-stop-word/stopword.txt
/kaggle/input/urdu-news-data/description.csv
/kaggle/input/urdu-news-data/headline.csv


In [46]:
# Import libraries
# Import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import re

In [58]:
from keras.layers import GRU

In [48]:
# Define maximum sequence length
MAX_SEQUENCE_LENGTH = 1000

In [49]:
# Load the CSV file
data = pd.read_csv('/kaggle/input/urdu-news-data/description.csv',engine='python',encoding='utf-8')

data = data.dropna()

In [50]:
stop_word_list = []
with open("/kaggle/input/urdu-stop-word/stopword.txt") as fin:
    for i in fin:
        stop_word_list.append(i.strip())
fin.close()

In [51]:
pattern = r'[a-zA-Z0-9!?:;"(,)]'
data['Description'] = data['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_word_list]))
data['Description'] = data['Description'].apply(lambda x: re.sub(pattern,'',x))

In [52]:
descriptions = data['Description'].tolist()
categories = data['Category'].tolist()

In [53]:
# Tokenize text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(descriptions)
sequences = tokenizer.texts_to_sequences(descriptions)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [54]:
# Encode categories
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(categories)
num_classes = len(label_encoder.classes_)

In [55]:
# Split data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(padded_sequences, integer_encoded_labels, test_size=0.2)

In [59]:
# Model definition
model = tf.keras.Sequential([
    Embedding(5000, 128, input_length=MAX_SEQUENCE_LENGTH),
    GRU(64, return_sequences=True),
    GRU(32),
    Dense(num_classes, activation="softmax"),
])

In [60]:
# Compile model
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)


In [61]:
# Train the model
model.fit(
    train_data,
    train_labels,
    epochs=10,
    validation_data=(test_data, test_labels)
)


Epoch 1/10
4431/4431 [==============================] - 385s 85ms/step - loss: 1.2144 - accuracy: 0.5373 - val_loss: 1.1155 - val_accuracy: 0.5835
Epoch 2/10
4431/4431 [==============================] - 344s 78ms/step - loss: 1.0472 - accuracy: 0.6029 - val_loss: 1.0852 - val_accuracy: 0.5844
Epoch 3/10
4431/4431 [==============================] - 342s 77ms/step - loss: 0.9931 - accuracy: 0.6172 - val_loss: 1.0796 - val_accuracy: 0.5850
Epoch 4/10
4431/4431 [==============================] - 342s 77ms/step - loss: 0.9499 - accuracy: 0.6284 - val_loss: 1.0885 - val_accuracy: 0.5829
Epoch 5/10
4431/4431 [==============================] - 342s 77ms/step - loss: 0.9077 - accuracy: 0.6386 - val_loss: 1.1148 - val_accuracy: 0.5768
Epoch 6/10
4431/4431 [==============================] - 342s 77ms/step - loss: 0.8628 - accuracy: 0.6496 - val_loss: 1.1450 - val_accuracy: 0.5697
Epoch 7/10
4431/4431 [==============================] - 341s 77ms/step - loss: 0.8169 - accuracy: 0.6606 - val_loss: 1

In [62]:
# Predict on test data
predictions = model.predict(test_data)

1108/1108 [==============================] - 32s 29ms/step


In [63]:
# Generate classification report
print("GRU Report")
y_pred = np.argmax(predictions, axis=1)
print(classification_report(test_labels, y_pred))

GRU Report
              precision    recall  f1-score   support

           0       0.07      0.02      0.03       182
           1       0.67      0.69      0.68      6256
           2       0.40      0.47      0.43      6383
           3       0.54      0.54      0.54      5108
           4       0.40      0.34      0.37      6505
           5       0.73      0.79      0.76      4929
           6       0.46      0.43      0.45      6086

    accuracy                           0.53     35449
   macro avg       0.47      0.47      0.46     35449
weighted avg       0.52      0.53      0.53     35449

